In [41]:
import warnings
warnings.filterwarnings("ignore")

import os
import logging
logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))

# loading packages
# basic + dates 
import numpy as np
import pandas as pd
from pandas import datetime

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
sns.set()
%matplotlib inline

##
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import mlflow

# custom functions
import sys
sys.path.append("/Users/java/Documents/10Acadamy/week - 3/Rossmann Pharmaceuticals")
from myscripts import util
from myscripts import plot

In [2]:
## Read Datas

store_file = '../data/store.csv'
test_file = '../data/test.csv'
train_file = '../data/train.csv'

In [3]:
train_df = pd.read_csv(train_file, parse_dates = True, low_memory = False)

In [4]:
test_df = pd.read_csv(test_file,parse_dates = True, low_memory = False)

### Preprocessing

In [5]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df = util.add_month_year(train_df)
# train_df.isnull().sum()

In [6]:
test_df['Date'] = pd.to_datetime(test_df['Date'])
test_df = util.add_month_year(test_df)
# test_df.isnull().sum()

In [7]:
test_df[(test_df['Open'] != 0) & (test_df['Open'] != 1)]
test_df["Open"].fillna(test_df['Open'].mode()[0], inplace=True)
test_df["Open"].value_counts()

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


1.0    35104
0.0     5984
Name: Open, dtype: int64

In [47]:
# train_df.info()

In [8]:
## Lable Date and StateHoiday Train 
lb_encode = LabelEncoder()
train_df['Date'] = lb_encode.fit_transform(train_df['Date'])
train_df['StateHoliday'] = lb_encode.fit_transform(train_df['StateHoliday'])
# train_df.info()

In [9]:
## Label encode for Test
test_df['Date'] = lb_encode.fit_transform(test_df['Date'])
test_df['StateHoliday'] = lb_encode.fit_transform(test_df['StateHoliday'])

In [42]:
# test_df.info()

In [12]:
scaler = MinMaxScaler()
scalled_train = scaler.fit_transform(train_df)
scalled_train_df = pd.DataFrame(data = scalled_train, columns=train_df.columns)
scalled_train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear
0,0.000000,0.666667,1.0,0.126664,0.075122,1.0,1.0,0.0,1.0,1.0,0.545455,1.0,0.588235
1,0.000898,0.666667,1.0,0.145941,0.084597,1.0,1.0,0.0,1.0,1.0,0.545455,1.0,0.588235
2,0.001795,0.666667,1.0,0.200091,0.111126,1.0,1.0,0.0,1.0,1.0,0.545455,1.0,0.588235
3,0.002693,0.666667,1.0,0.336815,0.202761,1.0,1.0,0.0,1.0,1.0,0.545455,1.0,0.588235
4,0.003591,0.666667,1.0,0.116050,0.075663,1.0,1.0,0.0,1.0,1.0,0.545455,1.0,0.588235


In [13]:
scaler = MinMaxScaler()
scalled_test = scaler.fit_transform(test_df)
scalled_test_df = pd.DataFrame(data = scalled_test, columns=test_df.columns)
scalled_test_df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear
0,0.000000,0.000000,0.5,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.533333,1.0
1,0.000024,0.001795,0.5,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.533333,1.0
2,0.000049,0.005386,0.5,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.533333,1.0
3,0.000073,0.006284,0.5,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.533333,1.0
4,0.000097,0.007181,0.5,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.533333,1.0


In [37]:
columns = ['DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear']

sample_size = 2000
testing_df = test_df[columns].sample(sample_size)

train_x = train_df[columns].sample(sample_size)
sales_y = train_df['Sales'].sample(sample_size)
customers_y = train_df['Customers'].sample(sample_size)

### Building models with sklearn pipelines

In [38]:
X_train, X_test, y_train, y_test = train_test_split(train_x, sales_y, test_size=0.3, random_state=123)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [39]:
score  = clf.score(X_test, y_test)
score

0.175

In [40]:
print(f"Prediction Score of the Model is {round(score * 100, 2)}%")

Prediction Score of the Model is 17.5%


### Choose a loss function

### Post Prediction analysis

### Serialize models

### Building model with deep learning

### Using MLFlow to serve the prediction